In [64]:
import huggingface
import pandas as pd
import torch
import numpy as np
from pyparsing import Empty
from torch.distributed.tensor import empty
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import datetime
from collections import defaultdict
import networkx as nx
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [60]:
data=pd.read_pickle("reddit_corpus.pkl")

Exception ignored in: <function tqdm.__del__ at 0x000001DDE9CD1CF0>
Traceback (most recent call last):
  File "C:\Users\Dell\PycharmProjects\Manu_factoid_GCN\.venv\lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "C:\Users\Dell\PycharmProjects\Manu_factoid_GCN\.venv\lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x000001DDE9CD1CF0>
Traceback (most recent call last):
  File "C:\Users\Dell\PycharmProjects\Manu_factoid_GCN\.venv\lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "C:\Users\Dell\PycharmProjects\Manu_factoid_GCN\.venv\lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x000001DDE9CD1C

In [62]:
data

,user_id,fake_news_spreader,documents,amounts,embedding_file,rn_amounts,fn_amounts,fn_rn_ratio,annotation,num_docs,...,4605,4606,4607,4608,4609,4610,4611,4612,4613,4614
0,ddf62a134f09db7a6056190d594bd41ff2f4cf04faf5db...,0,"[(gloapg3, The fact that this has to be explai...","(24, 1)",doc_embeddings_1.txt,24,1,0.041667,"[[], [], [], [], [(nbcnews.com, 0, ['LEFT_CENT...",317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a78d753b103a2a4310ad6d1d49723733a6e4744bbf1beb...,0,"[(fyfpnj4, Shelby County vs Holder. One of the...","(21, 0)",doc_embeddings_1.txt,21,0,0.000000,"[[], [], [], [], [], [], [(nbcnews.com, 0, ['L...",392,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19a734ee97b484a1a6c9625a1a3a0b5ddbabde3679ee2c...,0,"[(gmpzs0h, I've heard audio of him saying he'd...","(63, 0)",doc_embeddings_1.txt,63,0,0.000000,"[[], [], [], [], [], [], [], [], [], [], [], [...",3266,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,d57699be1824c59714fb9a799b7b5b670227b07e2bde00...,0,"[(gghar1g, No. The democrats and with as many ...","(11, 1)",doc_embeddings_1.txt,11,1,0.090909,"[[], [], [], [], [], [], [], [], [], [], [], [...",626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26d0132074c91843483fb0f8b0378083f876d748be172b...,1,"[(fyropxg, So, two weeks at Club Fed and 5 las...","(12, 5)",doc_embeddings_1.txt,12,5,0.416667,"[[], [], [], [], [], [], [], [], [], [(msn.com...",142,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610,e404255fd2be1d1e3c50c80f3ab21ec1be465c15d89541...,0,"[(fz673tq, Somebody literally said that to me ...","(5, 0)",doc_embeddings_5.txt,5,0,0.000000,"[[], [], [], [], [], [], [], [], [], [], [], [...",896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4611,e68fe2165fb6db674ae03dca13f1d14830d2a44853ebb2...,0,"[(gkezeid, Please dont insult sex workers. The...","(5, 0)",doc_embeddings_5.txt,5,0,0.000000,"[[], [], [], [], [], [], [], [], [], [], [], [...",331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4612,cee6bf50d84a8287e957feed646da5aae981c53076acd9...,0,"[(fvlkgw8, 7,000 max. Womp womp., 2020-06-22 0...","(5, 0)",doc_embeddings_5.txt,5,0,0.000000,"[[], [], [], [], [], [], [], [], [], [], [], [...",963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4613,95157a1c2c81dffdfacd4cfeb9fb62d82b6be831852ad3...,0,"[(fztr5v7, Just imagine how tired Cucker Tarls...","(2, 0)",doc_embeddings_5.txt,2,0,0.000000,"[[], [], [], [], [], [], [], [], [], [], [], [...",657,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
data=data[['user_id','fake_news_spreader',"documents","rn_amounts",'fn_amounts',"fn_rn_ratio",'pb_factor','sb_factor','troll_factor',"factual_factor"]]

In [131]:
data

,user_id,fake_news_spreader,documents,rn_amounts,fn_amounts,fn_rn_ratio,pb_factor,sb_factor,troll_factor,factual_factor
0,ddf62a134f09db7a6056190d594bd41ff2f4cf04faf5db...,0,"[(gloapg3, The fact that this has to be explai...",24,1,0.041667,-0.826087,NaN,NaN,1.666667
1,a78d753b103a2a4310ad6d1d49723733a6e4744bbf1beb...,0,"[(fyfpnj4, Shelby County vs Holder. One of the...",21,0,0.000000,-0.952381,NaN,NaN,2.000000
2,19a734ee97b484a1a6c9625a1a3a0b5ddbabde3679ee2c...,0,"[(gmpzs0h, I've heard audio of him saying he'd...",63,0,0.000000,-0.508197,1.0,NaN,2.015873
3,d57699be1824c59714fb9a799b7b5b670227b07e2bde00...,0,"[(gghar1g, No. The democrats and with as many ...",11,1,0.090909,-0.333333,NaN,NaN,1.500000
4,26d0132074c91843483fb0f8b0378083f876d748be172b...,1,"[(fyropxg, So, two weeks at Club Fed and 5 las...",12,5,0.416667,-1.083333,NaN,NaN,0.583333
...,...,...,...,...,...,...,...,...,...,...
4610,e404255fd2be1d1e3c50c80f3ab21ec1be465c15d89541...,0,"[(fz673tq, Somebody literally said that to me ...",5,0,0.000000,-0.200000,NaN,NaN,1.800000
4611,e68fe2165fb6db674ae03dca13f1d14830d2a44853ebb2...,0,"[(gkezeid, Please dont insult sex workers. The...",5,0,0.000000,-1.000000,NaN,NaN,1.800000
4612,cee6bf50d84a8287e957feed646da5aae981c53076acd9...,0,"[(fvlkgw8, 7,000 max. Womp womp., 2020-06-22 0...",5,0,0.000000,-0.200000,NaN,NaN,2.600000
4613,95157a1c2c81dffdfacd4cfeb9fb62d82b6be831852ad3...,0,"[(fztr5v7, Just imagine how tired Cucker Tarls...",2,0,0.000000,0.000000,NaN,NaN,2.000000


In [6]:
pip install huggingface-cli

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

C:\Users\Dell\PycharmProjects\Manu_factoid_GCN\.venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--meta-llama--Llama-3.2-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.45s/it]


In [35]:
data["documents"][0][4][4]

[('nbcnews.com', 0, ['LEFT_CENTER'], 'HIGH')]

In [36]:
data["documents"][0][4][4][0][3]

'HIGH'

In [105]:
def create_fake_news_prompt(post_text):
    return f"""You are an AI specialized in detecting misinformation and fake news.
    
Post: {post_text}

Rate this post on a scale from 1 to 3, where:
1: Contains no misleading information or false claims
2: unsure or post doesn't make a factual argument, only expressing opinion
3: Likely contains misleading information or false claims

Provide only the numerical rating here:



And then a short explanation here:
"""

In [156]:
def classify_post_with_llm(post_text, max_length=1000):
    if len(post_text) > max_length:
        post_text = post_text[:max_length] + "..."
    
    prompt = create_fake_news_prompt(post_text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            temperature=0.2,  #mozebi ke se smeni, za da ima po deterministicki izlezi
            do_sample=False
        )
        
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    parts=response.split("And then a short explanation here:")
    print(parts[1])
    try:
        for char in parts[1]:
            if char.isdigit() and int(char) in [1, 2, 3]:
                return int(char)
        return 2
    except:
        return 2

In [157]:
def reliability_to_score(reliability):
    reliability_map = {
        'VERY_HIGH': 1,  
        'HIGH': 2,      
        'MIXED': 3,     
        'LOW': 4, 
        'VERY_LOW': 5
    }
    
    if reliability is None:
        return 3  
    
    reliability = reliability.upper()
    return reliability_map.get(reliability, 3)


In [158]:
def extract_source_reliability(post_data):
    if len(post_data) < 5 or not post_data[4]:
        return 3 
    
    sources = post_data[4]
    if not sources:
        return 3
    
    reliability_scores = []
    for source in sources:
        if len(source) >= 4:
            reliability = source[3]
            reliability_scores.append(reliability_to_score(reliability))
    
    if not reliability_scores:
        return 3
    
    return sum(reliability_scores)

In [159]:
def is_fake_news(post_data, llm_weight=0.7, reliability_weight=0.3):
    if len(post_data) < 2:
        return False
    
    post_text = post_data[1]
    
    llm_score = classify_post_with_llm(post_text)
    
    reliability_score = extract_source_reliability(post_data)
    
    if llm_score == 3:
        llm_score=5
    elif llm_score == 2:
        llm_score=3
    # Combined weighted score (higher score = more likely fake news)
    combined_score = (llm_weight * llm_score) + (reliability_weight * reliability_score)
    
    # Threshold for classifying as fake news (can be tuned)
    fake_news_threshold = 3.5
    
    return combined_score >= fake_news_threshold, combined_score

# Batch Processing users' posts

In [160]:
def sample_users(dataset, sample_size=100, random_seed=42):
    # Set seed for reproducibility
    np.random.seed(random_seed)
    
    # Get unique user IDs
    all_user_ids = dataset['user_id'].unique().tolist()
    
    # Calculate sample size (ensure it's not larger than available users)
    sample_size = min(sample_size, len(all_user_ids))
    
    # Randomly sample users
    sampled_user_ids = np.random.choice(all_user_ids, size=sample_size, replace=False)
    
    return sampled_user_ids.tolist()

In [161]:
user_ids = sample_users(data, sample_size=50)

In [133]:
user_data=data[data['user_id']=="ddf62a134f09db7a6056190d594bd41ff2f4cf04faf5db32406bbec473f46935"]

In [134]:
user_data

,user_id,fake_news_spreader,documents,rn_amounts,fn_amounts,fn_rn_ratio,pb_factor,sb_factor,troll_factor,factual_factor
0,ddf62a134f09db7a6056190d594bd41ff2f4cf04faf5db...,0,"[(gloapg3, The fact that this has to be explai...",24,1,0.041667,-0.826087,NaN,NaN,1.666667


In [145]:
user_documents = user_data.iloc[0]['documents']
for i in user_documents:
    if i[4]:
        print(i[4])

[('nbcnews.com', 0, ['LEFT_CENTER'], 'HIGH')]
[('abcnews.go.com', 0, ['LEFT_CENTER'], 'HIGH')]
[('ballotpedia.org', 0, ['LEAST_BIASED'], 'VERY_HIGH'), ('ballotpedia.org', 0, ['LEAST_BIASED'], 'VERY_HIGH'), ('nytimes.com', 0, ['LEFT_CENTER'], 'HIGH')]
[('axios.com', 0, ['LEFT_CENTER'], 'HIGH'), ('desmoinesregister.com', 0, ['LEAST_BIASED'], 'HIGH'), ('usatoday.com', 0, ['LEFT_CENTER'], 'HIGH'), ('thehill.com', 0, ['LEAST_BIASED'], 'MOSTLY_FACTUAL'), ('nytimes.com', 0, ['LEFT_CENTER'], 'HIGH')]
[('usatoday.com', 0, ['LEFT_CENTER'], 'HIGH')]
[('fivethirtyeight.com', 0, ['LEFT_CENTER'], 'HIGH')]
[('washingtonpost.com', 0, ['LEFT_CENTER'], 'MOSTLY_FACTUAL')]
[('washingtonpost.com', 0, ['LEFT_CENTER'], 'MOSTLY_FACTUAL')]
[('washingtonpost.com', 0, ['LEFT_CENTER'], 'MOSTLY_FACTUAL')]
[('politico.com', 0, ['LEFT_CENTER'], 'HIGH')]
[('thehill.com', 0, ['LEAST_BIASED'], 'MOSTLY_FACTUAL'), ('newsweek.com', 0, ['LEFT_CENTER'], 'MOSTLY_FACTUAL')]
[('thehill.com', 0, ['LEAST_BIASED'], 'MOSTLY_FACTUA

In [ ]:
batch_size=10
max_posts_per_user=120

In [162]:
user_results = {}

for i in tqdm(range(0, len(user_ids), batch_size)):
    batch_users = user_ids[i:i+batch_size]
    
    for user_id in batch_users:
        user_data = data[data['user_id'] == user_id]
        
        if 'documents' not in user_data.columns or len(user_data) == 0:
            continue
            
        user_documents = user_data.iloc[0]['documents']
        posts_with_urls=[]
        posts_without_urls=[]
        for document in user_documents:
            if document[4]:
                posts_with_urls.append(document)
            else:
                posts_without_urls.append(document)
        
        
        posts_to_analyze = posts_with_urls[:min(len(posts_with_urls), max_posts_per_user)]
        if len(posts_with_urls) < max_posts_per_user:
            posts_to_analyze.extend(posts_without_urls[:min(len(user_documents), max_posts_per_user-len(posts_with_urls))])
        
        fake_news_count = 0
        fake_news_scores = []
        post_results = []
        
        for post in tqdm(posts_to_analyze, desc=f"User {user_id} posts", leave=False):            
            is_fake, score = is_fake_news(post)
            post_results.append({
                'post_id': post[0] if len(post) > 0 else None,
                'text': post[1] if len(post) > 1 else None,
                'timestamp': post[2] if len(post) > 2 else None,
                'subreddit': post[3] if len(post) > 3 else None,
                'is_fake_news': is_fake,
                'fake_news_score': score
            })
            
            if is_fake:
                fake_news_count += 1
            fake_news_scores.append(score)
        
        user_results[user_id] = {
            'fake_news_count': fake_news_count,
            'total_posts_analyzed': len(posts_to_analyze),
            'fake_news_ratio': fake_news_count / len(posts_to_analyze) if posts_to_analyze else 0,
            'avg_fake_news_score': np.mean(fake_news_scores) if fake_news_scores else 0,
            'post_details': post_results
        }



  0%|          | 0/5 [00:00<?, ?it/s]

User 67a0a400cb25f3034e9ab23ebb26b2becdfd795ef81b43495ce373ef7df527fe posts:   0%|          | 0/120 [00:00<?, ?it/s]C:\Users\Dell\PycharmProjects\Manu_factoid_GCN\.venv\lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\Dell\PycharmProjects\Manu_factoid_GCN\.venv\lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


User 67a0a400cb25f3034e9ab23ebb26b2becdfd795ef81b43495ce373ef7df527fe posts:   1%|          | 1/120 [00:0


The post is rated 1, as it appears




User 67a0a400cb25f3034e9ab23ebb26b2becdfd795ef81b43495ce373ef7df527fe posts:   2%|▏         | 2/120 [00:18<18:31,  9.42s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



The post is rated 1, as it appears




User 67a0a400cb25f3034e9ab23ebb26b2becdfd795ef81b43495ce373ef7df527fe posts:   2%|▎         | 3/120 [00:28<18:33,  9.52s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...


The post is rated 1, as it reports




User 6435a59b43e443404de4651326f8da78b2356b493124c36e7463d67e84c68223 posts:  99%|█████████▉| 119/120 [17:50<00:09,  9.07s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



This post is rated 1, as it appears




User 6435a59b43e443404de4651326f8da78b2356b493124c36e7463d67e84c68223 posts: 100%|██████████| 120/120 [17:59<00:00,  8.97s/it]

                                                                                                                              
100%|██████████| 5/5 [15:17:39<00:00, 11011.86s/it]  


This post is rated 3, likely contains misleading


In [163]:
import pickle

In [164]:
with open('checkpoint_file.pkl', 'wb') as f:
        pickle.dump(user_results, f)